# Imports

In [4]:
import pandas as pd
import numpy as np
import bcolz
import random
from importlib import reload
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

Using Theano backend.


# Precalculate ResNet50 vecs

In [2]:
resnet_model = ResNet50(include_top=False, input_shape=(3, 224, 224))

In [3]:
def save_array(fname, arr): 
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname): 
    return bcolz.open(fname)[:]

def save_resnet_vecs(vec_fname, img_name_fname, folder):
    gen = ImageDataGenerator(preprocessing_function=preprocess_input)
    batches = gen.flow_from_directory(folder, 
                                      target_size=(224, 224), 
                                      batch_size=64, 
                                      class_mode=None,
                                      shuffle=False)
    vecs = resnet_model.predict_generator(batches, steps=(batches.samples + batches.batch_size) 
                                          // batches.batch_size)
    save_array(vec_fname, vecs)
    save_array(img_name_fname, batches.filenames)
    
def load_resnet_vecs(vec_fname, img_name_fname):
    return load_array(img_name_fname), load_array(vec_fname)

In [ ]:
save_resnet_vecs('vecs', 'img_name', '../data/labeled')

# Finetune ResNet50

## Load data

In [5]:
fnames, vecs = load_resnet_vecs('vecs', 'img_name')
df = pd.read_csv('../data/labels.csv')

labels = np.array([df[df['imgs'] == x[5:]].iat[0, 1] for x in fnames])
np.random.seed(123)

## Training single neuron on top of resnet vectors

### Estimating validation error stability

In [222]:
errors = []
np.random.seed(123)
for i in range(50):
    idxs = np.arange(vecs.shape[0])
    np.random.shuffle(idxs)
    x_train = vecs[idxs][:-20]
    y_train = labels[idxs][:-20]
    x_test = vecs[idxs][-20:]
    y_test = labels[idxs][-20:]
    inp = Input(shape=(2048, 1, 1))
    x = Flatten()(inp)
    x = Dense(1)(x)
    model = Model(inputs=inp, outputs=x)
    batch_size = 16
    model.compile(optimizer=Adam(lr=0.001), loss='mse')
    model.fit(x_train, y_train, batch_size=batch_size, epochs=30, verbose=0)
    errors.append(model.evaluate(x_test, y_test, verbose=0))

In [223]:
np.mean(errors), np.std(errors)

(1.9290200293064117, 0.98324587074633618)

### Training single model

In [195]:
batch_size = 32
model.compile(optimizer=Adam(lr=0.001), loss='mse')
model.fit(vecs[:-25], labels[:-25], validation_data=(vecs[-25:], labels[-25:]), batch_size=batch_size, epochs=30)

Train on 175 samples, validate on 25 samples
Epoch 1/30
175/175 [==============================] - 0s - loss: 8.6968 - val_loss: 7.4721
Epoch 2/30
175/175 [==============================] - 0s - loss: 6.5714 - val_loss: 3.1279
Epoch 3/30
175/175 [==============================] - 0s - loss: 4.7415 - val_loss: 3.6431
Epoch 4/30
175/175 [==============================] - 0s - loss: 4.0315 - val_loss: 3.3068
Epoch 5/30
175/175 [==============================] - 0s - loss: 3.8258 - val_loss: 2.7392
Epoch 6/30
175/175 [==============================] - 0s - loss: 3.3161 - val_loss: 2.4167
Epoch 7/30
175/175 [==============================] - 0s - loss: 3.1926 - val_loss: 2.2984
Epoch 8/30
175/175 [==============================] - 0s - loss: 3.1278 - val_loss: 2.3565
Epoch 9/30
175/175 [==============================] - 0s - loss: 2.8444 - val_loss: 2.0184
Epoch 10/30
175/175 [==============================] - 0s - loss: 2.6886 - val_loss: 2.0313
Epoch 11/30
175/175 [=======================

### Checking predictions

In [199]:
model.evaluate(vecs[-25:], labels[-25:])

25/25 [==============================] - 0s


1.5130581855773926

In [145]:
preds = np.ravel(model.predict(vecs[-25:]))
pd.DataFrame({'abs':np.abs(preds - labels[-25:]), 
              'fname': fnames[-25:], 'label': labels[-25:], 'predict': preds}).sort_values('abs')


,abs,fname,label,predict
15,0.004653,imgs/20170920163402.jpg,3,3.004653
22,0.017748,imgs/20170906201015.jpg,3,3.017748
24,0.026531,imgs/20170912210045.jpg,0,0.026531
12,0.128895,imgs/20170921101101.jpg,2,2.128895
17,0.181843,imgs/20170918171101.jpg,3,3.181843
6,0.244945,imgs/20170911184045.jpg,4,4.244945
8,0.295847,imgs/20170907131701.jpg,0,0.295847
13,0.297246,imgs/20170911182945.jpg,4,4.297246
0,0.300162,imgs/20170918195731.jpg,5,5.300162
10,0.308294,imgs/20170913141230.jpg,2,1.691706


### MSE for prediction mean target

In [6]:
np.mean((np.square(labels - np.mean(labels))))

5.2555999999999994

## Train linear model on top of resnet vectors

In [310]:
vecs_lin = vecs.reshape(200, 2048)

In [344]:
regr = linear_model.Lasso(alpha = 0.05)
scores = cross_val_score(regr, vecs_lin, labels, cv=5, scoring='neg_mean_squared_error')
scores, np.mean(scores), np.std(scores)

(array([-1.48226493, -2.12800788, -1.31995179, -3.30496117, -1.11939934]),
 -1.8709170235018568,
 0.79271309996396244)

In [347]:
regr = linear_model.Lasso(alpha = 0.1)
scores = cross_val_score(regr, vecs_lin, labels, cv=5, scoring='neg_mean_squared_error')
scores, np.mean(scores), np.std(scores)

(array([-1.73242223, -2.06044628, -1.27759137, -3.6614439 , -1.11374428]),
 -1.9691296113420385,
 0.90974787362679765)

In [272]:
regr = linear_model.Lasso(alpha = 0.07)
scores = cross_val_score(regr, vecs_lin, labels, cv=5, scoring='neg_mean_squared_error')
scores, np.mean(scores), np.std(scores)

(array([-1.58147576, -2.10362221, -1.29454972, -3.42746844, -1.09528648]),
 -1.9004805205266286,
 0.83548934958463994)

In [359]:
regr = linear_model.Ridge(alpha = 200)
scores = cross_val_score(regr, vecs_lin, labels, cv=5, scoring='neg_mean_squared_error')
scores, np.mean(scores), np.std(scores)

(array([-1.32685227, -2.11493647, -1.20068443, -3.19762256, -1.125603  ]),
 -1.7931397428766136,
 0.78614982423876145)

In [360]:
regr.fit(vecs_lin, labels)

Ridge(alpha=200, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [361]:
mean_squared_error(labels, regr.predict(vecs_lin))

1.0110379412439496

## Train SVM regression on top of resnet vectors 

In [358]:
svr = svm.SVR(C=1, kernel='poly', degree=2)

scores = cross_val_score(svr, vecs_lin, labels, cv=5, scoring='neg_mean_squared_error')
scores, np.mean(scores), np.std(scores)

(array([-2.54821538, -2.88886956, -1.2938378 , -5.1070622 , -1.96383601]),
 -2.7603641874476441,
 1.2925686888970245)

In [328]:
svr.fit(vecs_lin, labels)
mean_squared_error(labels, svr.predict(vecs_lin))

1.1060901904407299

## Training on vectors from intermediate Conv layer

In [283]:
resnet_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 64, 112, 112)  9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, 112, 112)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 64, 112, 112)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

bn3a_branch2b (BatchNormalizatio (None, 128, 28, 28)   512         res3a_branch2b[0][0]             
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 128, 28, 28)   0           bn3a_branch2b[0][0]              
____________________________________________________________________________________________________
res3a_branch2c (Conv2D)          (None, 512, 28, 28)   66048       activation_12[0][0]              
____________________________________________________________________________________________________
res3a_branch1 (Conv2D)           (None, 512, 28, 28)   131584      activation_10[0][0]              
____________________________________________________________________________________________________
bn3a_branch2c (BatchNormalizatio (None, 512, 28, 28)   2048        res3a_branch2c[0][0]             
___________________________________________________________________________________________

bn4a_branch2a (BatchNormalizatio (None, 256, 14, 14)   1024        res4a_branch2a[0][0]             
____________________________________________________________________________________________________
activation_23 (Activation)       (None, 256, 14, 14)   0           bn4a_branch2a[0][0]              
____________________________________________________________________________________________________
res4a_branch2b (Conv2D)          (None, 256, 14, 14)   590080      activation_23[0][0]              
____________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizatio (None, 256, 14, 14)   1024        res4a_branch2b[0][0]             
____________________________________________________________________________________________________
activation_24 (Activation)       (None, 256, 14, 14)   0           bn4a_branch2b[0][0]              
___________________________________________________________________________________________

res4e_branch2b (Conv2D)          (None, 256, 14, 14)   590080      activation_35[0][0]              
____________________________________________________________________________________________________
bn4e_branch2b (BatchNormalizatio (None, 256, 14, 14)   1024        res4e_branch2b[0][0]             
____________________________________________________________________________________________________
activation_36 (Activation)       (None, 256, 14, 14)   0           bn4e_branch2b[0][0]              
____________________________________________________________________________________________________
res4e_branch2c (Conv2D)          (None, 1024, 14, 14)  263168      activation_36[0][0]              
____________________________________________________________________________________________________
bn4e_branch2c (BatchNormalizatio (None, 1024, 14, 14)  4096        res4e_branch2c[0][0]             
___________________________________________________________________________________________

res5c_branch2b (Conv2D)          (None, 512, 7, 7)     2359808     activation_47[0][0]              
____________________________________________________________________________________________________
bn5c_branch2b (BatchNormalizatio (None, 512, 7, 7)     2048        res5c_branch2b[0][0]             
____________________________________________________________________________________________________
activation_48 (Activation)       (None, 512, 7, 7)     0           bn5c_branch2b[0][0]              
____________________________________________________________________________________________________
res5c_branch2c (Conv2D)          (None, 2048, 7, 7)    1050624     activation_48[0][0]              
____________________________________________________________________________________________________
bn5c_branch2c (BatchNormalizatio (None, 2048, 7, 7)    8192        res5c_branch2c[0][0]             
___________________________________________________________________________________________

In [ ]:
def save_array(fname, arr): 
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname): 
    return bcolz.open(fname)[:]

def save_resnet_vecs(vec_fname, img_name_fname, folder):
    gen = ImageDataGenerator(preprocessing_function=preprocess_input)
    batches = gen.flow_from_directory(folder, 
                                      target_size=(224, 224), 
                                      batch_size=64, 
                                      class_mode=None,
                                      shuffle=False)
    inp = resnet_model.input
    out = resnet_model.get_layer('res5c_branch2c').output

    model = Model(inputs=inp, outputs=out)
    vecs = model.predict_generator(batches, steps=(batches.samples + batches.batch_size) 
                                          // batches.batch_size)
    save_array(vec_fname, vecs)
    save_array(img_name_fname, batches.filenames)
    
def load_resnet_vecs(vec_fname, img_name_fname):
    return load_array(img_name_fname), load_array(vec_fname)
    
save_resnet_vecs('vecs_inter', 'img_name_inter', '../data/labeled')

In [ ]:
fnames, vecs = load_resnet_vecs('vecs_inter', 'img_name_inter')
df = pd.read_csv('../data/labels.csv')

labels = np.array([df[df['imgs'] == x[5:]].iat[0, 1] for x in fnames])
np.random.seed(123)

In [291]:
vecs_lin = vecs.reshape(200, 100352)

In [294]:
regr = linear_model.Ridge(alpha = 1)
scores = cross_val_score(regr, vecs_lin, labels, cv=5, scoring='neg_mean_squared_error')
scores, np.mean(scores), np.std(scores)

(array([-1.20611439, -2.17058301, -1.09476578, -3.03697043, -1.55469707]),
 -1.8126261356719566,
 0.71789406796078514)

# Training LR on image pixels

In [298]:
gen = ImageDataGenerator(preprocessing_function=preprocess_input)
batches = gen.flow_from_directory('../data/labeled', 
                                  target_size=(224, 224), 
                                  batch_size=200, 
                                  class_mode=None,
                                  shuffle=False)
data = next(batches)

Found 200 images belonging to 1 classes.


In [299]:
data.shape

(200, 3, 224, 224)

In [300]:
vecs_lin = data.reshape(200, 150528)

In [303]:
regr = linear_model.Ridge(alpha = 100)
scores = cross_val_score(regr, vecs_lin, labels, cv=10, scoring='neg_mean_squared_error')
scores, np.mean(scores), np.std(scores)

/home/nighturs/Projects/pool-cnn/env/lib/python3.6/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 4.637792649919703e-10
  ' condition number: {}'.format(rcond), RuntimeWarning)
/home/nighturs/Projects/pool-cnn/env/lib/python3.6/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 5.387444379501005e-10
  ' condition number: {}'.format(rcond), RuntimeWarning)
/home/nighturs/Projects/pool-cnn/env/lib/python3.6/site-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 7.738502061371832e-10
  ' condition number: {}'.format(rcond), RuntimeWarning)
/home/nighturs/Projects/pool-cnn/env/lib/python3.6/site-packages/scipy/li

(array([-1.39252937, -2.35484155, -2.69914397, -3.24689025, -1.09135875,
        -1.07962793, -5.26584595, -4.0765285 , -1.58352768, -1.93104581]),
 -2.472133977691076,
 1.3091219837137371)

# Area averaging architecture

In [2]:
fnames, vecs = load_resnet_vecs('vecs', 'img_name')
df = pd.read_csv('../data/labels.csv')

labels = np.array([df[df['imgs'] == x[5:]].iat[0, 1] for x in fnames])
np.random.seed(123)

gen = ImageDataGenerator(preprocessing_function=preprocess_input)
batches = gen.flow_from_directory('../data/labeled', 
                                  target_size=(800, 1280), 
                                  batch_size=200, 
                                  class_mode=None,
                                  shuffle=False)
data = next(batches)

Found 200 images belonging to 1 classes.


In [ ]:
inp = Input((3,800,1280))
x = Conv2D(1, (1, 1), padding='same', activation='relu')(inp)
x = AveragePooling2D((20, 20))(x)
x = Flatten()(x)
x = Dense(1)(x)
model = Model(inp, x)
model.compile(optimizer='adam', loss='mse')
model.summary()

model.fit(data[:-20], labels[:-20], validation_data=(data[-20:], labels[-20:]), batch_size=64, epochs=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 800, 1280)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 800, 1280)      4         
_________________________________________________________________
average_pooling2d_1 (Average (None, 1, 40, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2560)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2561      
Total params: 2,565
Trainable params: 2,565
Non-trainable params: 0
_________________________________________________________________
Train on 180 samples, validate on 20 samples
Epoch 1/100
180/180 [==============================] - 6s - loss: 16.7012 - val_loss: 16.2308
E